In [30]:
import pandas
from sklearn import linear_model
from sklearn.metrics import *

axis = ["SMBFactor", "HMLFactor", "RMWFactor",
        "CMAFactor", "MoMFactor", "Market"]
df = pandas.read_excel("Factors (with market & industries).xlsx", "Factors")

def calculateRegression(x, y):
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    yPredict = regr.predict(x)
    return regr, yPredict


def prepareRegressionDF():
    regressionDF = {
        "month": []
    }
    for i in axis:
        regressionDF[i] = []
    regressionDF["IndustryYield"]= []
    regressionDF["R2"]= []
    regressionDF["intercept"]= []
    for i in axis:
        regressionDF["coefficient_" + i] = []
    return regressionDF


def fillRegressionDF(month,IndustryYield,factors,regressionDF,r2, intercept, coef_):
    regressionDF["month"].append(month)
    regressionDF["IndustryYield"].append(IndustryYield)
    for i in range(len(factors)):
      regressionDF[axis[i]].append(factors[i])
    regressionDF["R2"].append(r2)
    regressionDF["intercept"].append(intercept)
    for i in axis:
        regressionDF["coefficient_" +
                     i].append(coef_[axis.index(i)])


def produceRegressionDF(x,y):
    regressionDF = prepareRegressionDF()
    for i in range(len(y)):
        factors = []
        for factor in axis:
          factors.append(df[factor][i])
        if i < rollingSize-1:
            fillRegressionDF(df["month"][i],y.iloc[i],factors,regressionDF,None, None, [None]*6)
            continue

        start = i - rollingSize + 1
        end = i
        regr, yPredict = calculateRegression(x[start:end], y[start:end])
        fillRegressionDF(df["month"][i],y.iloc[i],factors,regressionDF,
            r2_score(y[start:end], yPredict), regr.intercept_, regr.coef_)
    return regressionDF


def saveDFToExcel(dataFrame,fileName,sheetName):
  regressionDF = pandas.DataFrame(dataFrame)
  regressionDF.to_excel(fileName + '.xlsx', sheet_name=sheetName,index=False)

def saveRegressionDF(isDisplay,regressionDF):
    regressionDF = pandas.DataFrame(regressionDF)
    regressionDF.to_excel('regressionOutput.xlsx', sheet_name='regression')
    if isDisplay:
        print(regressionDF[rollingSize-1-1:rollingSize-1+4])




rollingSize = 36
x = df[axis]
# y = df['IndustryExcessReturn']
industryCount = 50
industryColumns = ["Industry" + str(x) for x in range(1,industryCount+1)]

# regressionDF = produceRegressionDF(x,y)
# saveRegressionDF(True,regressionDF)


In [31]:
print(y.iloc[1])

-0.08


In [32]:

industryCount = 50
industryColumns = ["Industry" + str(x) for x in range(1,industryCount+1)]
print(industryCount)
print(industryColumns)
regressionOutput = {}
for i in industryColumns:
  y = df[i]
  regressionOutput[i] = produceRegressionDF(x,y)




50
['Industry1', 'Industry2', 'Industry3', 'Industry4', 'Industry5', 'Industry6', 'Industry7', 'Industry8', 'Industry9', 'Industry10', 'Industry11', 'Industry12', 'Industry13', 'Industry14', 'Industry15', 'Industry16', 'Industry17', 'Industry18', 'Industry19', 'Industry20', 'Industry21', 'Industry22', 'Industry23', 'Industry24', 'Industry25', 'Industry26', 'Industry27', 'Industry28', 'Industry29', 'Industry30', 'Industry31', 'Industry32', 'Industry33', 'Industry34', 'Industry35', 'Industry36', 'Industry37', 'Industry38', 'Industry39', 'Industry40', 'Industry41', 'Industry42', 'Industry43', 'Industry44', 'Industry45', 'Industry46', 'Industry47', 'Industry48', 'Industry49', 'Industry50']


In [33]:
import time
writer = pandas.ExcelWriter("regressionOutput.xlsx", engine = 'xlsxwriter')
for i in industryColumns:
  regressionDF = pandas.DataFrame(regressionOutput[i])
  regressionDF.to_excel(writer, sheet_name = i)
  
writer.save()
time.sleep(3)
writer.close()



/home/user/anaconda3/lib/python3.8/site-packages/xlsxwriter/workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [36]:
acceptedIndustriesList = []
for monthIndex in range(rollingSize, len(df)):
  allIntercepts = []
  for i in industryColumns:
    allIntercepts.append(regressionOutput[i]["intercept"][monthIndex])
  allIntercepts.sort()
  lastAcceptable = allIntercepts[int(industryCount * -0.1)]
  acceptedIndustries = []
  for i in industryColumns:
    if regressionOutput[i]["intercept"][monthIndex]>= lastAcceptable and regressionOutput[i]["intercept"][monthIndex]> 0:
      acceptedIndustries.append(i)
  acceptedIndustriesList.append(acceptedIndustries)

print(acceptedIndustriesList[:2])

[['Industry17', 'Industry23', 'Industry35', 'Industry36', 'Industry43'], ['Industry8', 'Industry17', 'Industry21', 'Industry23', 'Industry36']]


In [40]:
strategyYield = 1
monthlyYieldHistory = {"month":[],"yield":[],"strategyYield":[]}
for monthIndex in range(rollingSize, len(df)-1):
  weight = 1/ int(industryCount * 0.1)
  monthYield = 0
  for acceptedIndustry in acceptedIndustriesList[monthIndex-rollingSize]:
      monthYield += weight* regressionOutput[acceptedIndustry]["IndustryYield"][monthIndex+1]
  strategyYield *= 1+monthYield
  monthlyYieldHistory["month"].append(df["month"][monthIndex+1])
  monthlyYieldHistory["yield"].append(monthYield)
  monthlyYieldHistory["strategyYield"].append(strategyYield)

print(strategyYield)

saveDFToExcel(monthlyYieldHistory,"SectorRotation6AxisNaive","Monthly yields")

0.4767599899373884
